In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

In [2]:
from data.dataset import VideoDataSet
from sklearn.preprocessing import LabelEncoder
import random
import numpy as np
train_dataset = VideoDataSet('train')
test_dataset = VideoDataSet('test')
le = LabelEncoder()
label = train_dataset.get_label()
le.fit(label)



In [3]:

def collate_fn(batch):
    frames = [b[0] for b in batch]
    masks = [torch.ones(b[0].shape[0]) for b in batch]
    frames = torch.nn.utils.rnn.pad_sequence(frames)
    masks = torch.nn.utils.rnn.pad_sequence(masks)
    frames = torch.transpose(frames, 0 , 1)
    masks = torch.transpose(masks, 0 , 1)
    item = le.transform([b[1] for b in batch])
    labels = torch.tensor(item)
    return (frames, masks), labels

batch_size = 8
class_num = len(label)
# train_data_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32, collate_fn = collate_fn)
# test_data_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32, collate_fn = collate_fn)
train_data_loader = torch.load('train_data.pt')
train_data_loader = torch.utils.data.DataLoader(train_data_loader.dataset,batch_size=batch_size, collate_fn = collate_fn)
test_data_loader = torch.load('test_data.pt')
test_data_loader = torch.utils.data.DataLoader(test_data_loader.dataset,batch_size=batch_size, collate_fn = collate_fn)

In [14]:
from tqdm import tqdm
def train(data_loader, model, loss_fn, optimizer, masking):
    model.train()
    # pbar = tqdm(range(len(data_loader)))
    correct = 0
    for batch, (X, y) in enumerate(data_loader):
        # pbar.update()
        frames, mask = X
        frames = frames.to('cuda')
        y = y.type(torch.int64)

        pred = model(frames)
        pred = pred.to('cpu')
        if masking:
            mask = torch.count_nonzero(mask,axis=-1) - 1
            mask = torch.reshape(mask, (mask.shape[0],1,1))
            mask = torch.tile(mask, (1,1,class_num))
            pred = torch.gather(pred, 1, mask)
            pred = torch.squeeze(pred)
        else:
            pred = pred[:,-1,:]
            
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    #     pbar.set_description(f'loss:{loss.item():<.3f} acc:{correct / (data_loader.batch_size*(batch+1)) * 100 :.3f}%')
    # del pbar

def test(data_loader, model, loss_fn, masking):
    pbar = tqdm(range(len(data_loader)))
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(data_loader):
            pbar.update()
            frames, mask = X
            frames = frames.to('cuda')
            y = y.type(torch.int64)

            pred = model(frames)
            pred = pred.to('cpu')
            if masking:
                mask = torch.count_nonzero(mask,axis=-1) - 1
                mask = torch.reshape(mask, (mask.shape[0],1,1))
                mask = torch.tile(mask, (1,1,class_num))
                pred = torch.gather(pred, 1, mask)
                pred = torch.squeeze(pred)
            else:
                pred = pred[:,-1,:]

            loss = loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            pbar.set_description(f'loss:{loss.item():<.3f} acc:{correct / (data_loader.batch_size*(batch+1)) * 100 :.3f}%')
    del pbar

In [18]:

import Model

random.seed(14)
np.random.seed(14)
torch.manual_seed(14)
torch.cuda.manual_seed(14)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


print("train with masking")
for t in (range(10)):
    size = len(test_data_loader)
    model = Model.CNNRNN(class_num)
    model.to('cuda')
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    for i in range(20):
        train(train_data_loader, model, loss_fn, optimizer, True)
    test(test_data_loader, model, loss_fn, True)


random.seed(14)
np.random.seed(14)
torch.manual_seed(14)
torch.cuda.manual_seed(14)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


print("train without masking")
for t in (range(10)):
    size = len(test_data_loader)
    model = Model.CNNRNN(class_num)
    model.to('cuda')
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    for i in range(20):
        train(train_data_loader, model, loss_fn, optimizer, False)
    test(test_data_loader, model, loss_fn, False) 

loss:1.130 acc:86.161%: 100%|██████████| 28/28 [00:00<00:00, 208.81it/s]


KeyboardInterrupt: 